<a href="https://colab.research.google.com/github/ErikaRochadeAraujo/WebScrapingQuintoAndar/blob/main/QuintoAndar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping na página do Quinto Andar



Primeiro vamos importar bibliotecas

Depois verificamos na página onde podemos ter informações de dicionário, se existe.

Achamos o link onde ele faz as requisições.

In [51]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
pd.set_option("display.max_columns", None)
response = requests.get('https://www.quintoandar.com.br/api/yellow-pages/v2/search?map[bounds_north]=-22.68&map[bounds_south]=-23.13&map[bounds_east]=-43.01&map[bounds_west]=-43.34&availability=any&occupancy=any&house_type=Apartamento&business_context=RENT&return=location&photos=12&relax_query=false')
data = json.loads(response.text)



In [31]:
response

<Response [200]>

Olhando na página do Quinto Andar, podemos notar em "inspecionar" que temos onde tiramos o caminho do dicionário, "hits" e depois "hits" novamente. 
Aplicando então esse filtro temos:


In [50]:
data['hits']['hits']

[{'_index': 'house',
  '_type': '_doc',
  '_id': '893562391',
  '_score': 0.09133476,
  '_source': {'location': {'lon': -43.3283327, 'lat': -22.86009}}},
 {'_index': 'house',
  '_type': '_doc',
  '_id': '893745490',
  '_score': 0.08975081,
  '_source': {'location': {'lon': -43.3356498, 'lat': -22.849646}}},
 {'_index': 'house',
  '_type': '_doc',
  '_id': '893232976',
  '_score': 0.08964636,
  '_source': {'location': {'lon': -43.3022041, 'lat': -22.9090829}}},
 {'_index': 'house',
  '_type': '_doc',
  '_id': '893061854',
  '_score': 0.08946731,
  '_source': {'location': {'lon': -43.3022041, 'lat': -22.9090829}}},
 {'_index': 'house',
  '_type': '_doc',
  '_id': '893562661',
  '_score': 0.08938849,
  '_source': {'location': {'lon': -43.3283327, 'lat': -22.86009}}},
 {'_index': 'house',
  '_type': '_doc',
  '_id': '893554450',
  '_score': 0.08915855,
  '_source': {'location': {'lon': -43.0683931, 'lat': -22.8279037}}},
 {'_index': 'house',
  '_type': '_doc',
  '_id': '893710411',
  '_sco

In [49]:
df_results = pd.DataFrame(data['hits']['hits'])

Olha só como ficou o resultado

In [47]:
df_results

,_index,_type,_id,_score,_source
0,house,_doc,893562391,0.091335,"{'location': {'lon': -43.3283327, 'lat': -22.8..."
1,house,_doc,893745490,0.089751,"{'location': {'lon': -43.3356498, 'lat': -22.8..."
2,house,_doc,893232976,0.089646,"{'location': {'lon': -43.3022041, 'lat': -22.9..."
3,house,_doc,893061854,0.089467,"{'location': {'lon': -43.3022041, 'lat': -22.9..."
4,house,_doc,893562661,0.089388,"{'location': {'lon': -43.3283327, 'lat': -22.8..."
...,...,...,...,...,...
2126,house,_doc,893083335,0.036952,"{'location': {'lon': -43.189341, 'lat': -22.97..."
2127,house,_doc,893709945,0.034496,"{'location': {'lon': -43.1746565, 'lat': -22.9..."
2128,house,_doc,893030940,0.034113,"{'location': {'lon': -43.1760455, 'lat': -22.9..."
2129,house,_doc,893658041,0.031933,"{'location': {'lon': -43.1829986, 'lat': -22.9..."


Agora com o Id vamos conseguir pegar a identificação de cada imóvel

In [52]:
def getDetails(id):
  response = requests.get('https://www.quintoandar.com.br/imovel/'+id).content
  soup = BeautifulSoup(response, 'html.parser')
  dict_details = json.loads(soup.find(id = '__NEXT_DATA__').string)
  return dict_details['props']['pageProps']['initialState']['house']['houseInfo']


In [ ]:
results = []

Aqui precisei de ficar rodando várias vezes a partir da linha q dava erro. Existe a hipótese de que o Quinto Andar bloqueia muitas requisições, devido a isso, sempre continuava no ponto que foi bloqueado.

Executava a linha de baixo assim ---> 'for id in df_results['_id'][len(results):]:'

In [66]:

for id in df_results['_id']:
  res = getDetails(id)
  results.append(res)

In [67]:
len(results)

2131

In [85]:
df = pd.DataFrame(results)

In [86]:
df.shape

(2131, 73)

In [87]:
df

,id,displayId,bedrooms,city,cityId,bathrooms,iptu,amenities,comfortCommodities,practicalityCommodities,area,installations,rentPrice,condoPrice,condoType,region,type,iptuType,totalCost,parkingSpaces,address,floor,homeProtection,remarks,acceptsPets,hasFurniture,isNearSubway,placesNearby,availabilityType,photos,relevantRegions,photospheres,specialConditions,offers,status,showUrlHasBeenCopiedSnackbar,error,loaded,loading,selectedMedia,video,promotions,condominium,forRent,forSale,salePrice,listings,businessContext,regionParameters,houseAgent,tenantServiceFee,closingStatus,rentalGuarantee,reservationAvailability,macroRegion,suites,houseVisitStatus,visitStatusReason,visitsUnavailable,isReserved,allowDirectOffer,lastPublishedDate,listingPublication,rentOnTermination,constructionYear,allowsCustomOffer,displayCustomOfferPreference,furniture,partnerId,daysToVacate,movingDate,awaitingAvailabilityReason,terminationDate
0,893562391,862391,2,Rio de Janeiro,1507.0,1,41,"[{'key': 'BANHEIRA_DE_HIDROMASSAGEM', 'text': ...","[{'key': 'BANHEIRA_DE_HIDROMASSAGEM', 'text': ...","[{'key': 'BOX', 'text': 'Box', 'value': 'NAO',...",50,"[{'key': 'PLAYGROUND', 'text': 'Playground', '...",1150,0,IncluidoNoAluguel,"{'name': 'Irajá', 'hub': {'loading': 'UNSET', ...",Apartamento,Normal,1231,1,"{'street': 'Rua Lima Drumond', 'neighborhood':...",1,15,,True,False,False,"[{'name': 'Escola Municipal Irmã Zélia', 'type...",IMMEDIATE,"[{'id': 22860816, 'subtitle': 'Sala', 'url': '...",[],[],[],"{'closingRate': 0, 'shouldHighlightIcon': True...",publicado,False,False,True,False,tenant/HouseMedia/PHOTOS,,[],{},True,True,180000.0,"[{'imovelId': 893562391, 'businessContext': 'S...",,"{'loaded': False, 'loading': False, 'RENT': {'...","{'isDialogOpen': False, 'alreadyOpenedByQueryP...",25,{'loading': False},"{'minValue': 287.5, 'maxValue': 862.5}",None,"{'id': 2711, 'name': 'RIO 09'}",0.0,ACCEPT_NEW,None,False,False,True,2022-02-22T00:51:38.000Z,stranded,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,893745490,1045490,2,Rio de Janeiro,1507.0,1,0,"[{'key': 'BANHEIRA_DE_HIDROMASSAGEM', 'text': ...","[{'key': 'BANHEIRA_DE_HIDROMASSAGEM', 'text': ...","[{'key': 'BOX', 'text': 'Box', 'value': 'NAO',...",111,"[{'key': 'PLAYGROUND', 'text': 'Playground', '...",950,0,IncluidoNoAluguel,"{'name': 'Irajá', 'hub': {'loading': 'UNSET', ...",Apartamento,NaoExiste,983,0,"{'street': 'Rua Caxambú', 'neighborhood': 'Ira...",1,13,"Excelente casa, segundo andar, cômodos amplos,...",True,False,True,[{'name': 'Escola Municipal Rosa Bettiato Zátt...,IMMEDIATE,"[{'id': 29149685, 'subtitle': 'Sala', 'url': '...",[],[],[Exclusivity],"{'closingRate': 0, 'shouldHighlightIcon': True...",publicado,False,False,True,False,tenant/HouseMedia/PHOTOS,,[],{},True,False,0.0,"[{'imovelId': 893745490, 'businessContext': 'R...",,"{'loaded': False, 'loading': False, 'RENT': {'...","{'isDialogOpen': False, 'alreadyOpenedByQueryP...",20,{'loading': False},"{'minValue': 237.5, 'maxValue': 712.5}",None,"{'id': 2711, 'name': 'RIO 09'}",0.0,ACCEPT_NEW,None,False,False,True,2022-08-19T12:27:47.000Z,not_stranded,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,893232976,532976,1,Rio de Janeiro,1507.0,1,50,"[{'key': 'BANHEIRA_DE_HIDROMASSAGEM', 'text': ...","[{'key': 'BANHEIRA_DE_HIDROMASSAGEM', 'text': ...","[{'key': 'BOX', 'text': 'Box', 'value': 'NAO',...",50,"[{'key': 'PLAYGROUND', 'text': 'Playground', '...",770,0,NaoExiste,"{'name': 'Engenho de Dentro', 'hub': {'loading...",Apartamento,Normal,846,0,"{'street': 'Rua Eulina Ribeiro', 'neighborhood...",3,10,,True,False,False,[{'name': 'Escola Especial Municipal Doutor Ul...,NaN,"[{'id': 10829404, 'subtitle': 'Sala', 'url': '...",[],[],[],"{'closingRate': 0, 'shouldHighlightIcon': True...",publicado,False,False,True,False,tenant/HouseMedia/PHOTOS,,[],"{'hashId': 'jpqmsmez7d', 'name': 'Não possui',...",True,False,0.0,"[{'imovelId': 893232976, 'businessContext': 'R...",,"{'loaded': False, 'loading': False, 'RENT': {'...","{'isDialogOpen': False, 'alreadyOpenedByQueryP...",16,{'

Salvando em um arquivo.csv

In [80]:
df.to_csv("scrap5andar.csv")

In [88]:
df.columns

Index(['id', 'displayId', 'bedrooms', 'city', 'cityId', 'bathrooms', 'iptu',
       'amenities', 'comfortCommodities', 'practicalityCommodities', 'area',
       'installations', 'rentPrice', 'condoPrice', 'condoType', 'region',
       'type', 'iptuType', 'totalCost', 'parkingSpaces', 'address', 'floor',
       'homeProtection', 'remarks', 'acceptsPets', 'hasFurniture',
       'isNearSubway', 'placesNearby', 'availabilityType', 'photos',
       'relevantRegions', 'photospheres', 'specialConditions', 'offers',
       'status', 'showUrlHasBeenCopiedSnackbar', 'error', 'loaded', 'loading',
       'selectedMedia', 'video', 'promotions', 'condominium', 'forRent',
       'forSale', 'salePrice', 'listings', 'businessContext',
       'regionParameters', 'houseAgent', 'tenantServiceFee', 'closingStatus',
       'rentalGuarantee', 'reservationAvailability', 'macroRegion', 'suites',
       'houseVisitStatus', 'visitStatusReason', 'visitsUnavailable',
       'isReserved', 'allowDirectOffer', 'lastP

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2131 entries, 0 to 2130
Data columns (total 73 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            2131 non-null   object 
 1   displayId                     2131 non-null   object 
 2   bedrooms                      2131 non-null   int64  
 3   city                          2131 non-null   object 
 4   cityId                        2124 non-null   float64
 5   bathrooms                     2131 non-null   object 
 6   iptu                          2131 non-null   int64  
 7   amenities                     2131 non-null   object 
 8   comfortCommodities            2131 non-null   object 
 9   practicalityCommodities       2131 non-null   object 
 10  area                          2131 non-null   int64  
 11  installations                 2131 non-null   object 
 12  rentPrice                     2131 non-null   int64  
 13  con